In [ ]:
import numpy as np
import pandas as pd
import os
jpg_list=os.listdir('../input/siim-covid19-resized-to-256px-jpg/train')
train_data_df=pd.DataFrame(jpg_list)
train_data_df=train_data_df.reset_index()
train_data_df['image_id']=train_data_df[0]
train_data_df['process_run']=train_data_df['index']+1
del train_data_df[0]
del train_data_df['index']
train_data_df

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
import sys
train_data_df['id']=train_data_df['image_id'].str.replace('.jpg','_image')
train_image_level=pd.read_csv('../input/siim-covid19-detection/train_image_level.csv')
train_study=pd.read_csv('../input/siim-covid19-detection/train_study_level.csv')
train_study['StudyInstanceUID']=train_study['id'].str.replace("_study", "")
del train_study['id']
train_data_df=pd.merge(train_data_df, train_image_level, how='inner')
train_data_df=pd.merge(train_data_df, train_study, how='inner')
train_data_df
del train_image_level
del train_study


var_dict={'Negative for Pneumonia': 1, 'Typical Appearance': 2, 'Indeterminate Appearance': 3, 'Atypical Appearance': 4}
train_data_df['categorical_train']=0
for key, value in var_dict.items():
    train_data_df['categorical_train']=train_data_df[[key, 'categorical_train']].apply(
        lambda x:value if x[key]==1 else x['categorical_train'], axis=1)

print(train_data_df[['Negative for Pneumonia', 'Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance', 'categorical_train']].drop_duplicates())
y_train=train_data_df['categorical_train'].to_numpy()
print(y_train.shape)
y_train

In [ ]:
# train_data_df['y_hot']=y_train.astype(str)
train_data_df['checkmul']=train_data_df['Negative for Pneumonia']+train_data_df['Typical Appearance']+train_data_df['Indeterminate Appearance']+train_data_df['Atypical Appearance']
train_data_df['checkmul'].max()

In [ ]:
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers

train_datagen = ImageDataGenerator(
        rescale=1 / 255.0,
        zoom_range=0.05,
        width_shift_range=0.05,
        height_shift_range=0.05,
        fill_mode="nearest",
        validation_split=0.20)

train_generator=train_datagen.flow_from_dataframe(
dataframe=train_data_df.loc[0:5333],
directory="../input/siim-covid19-resized-to-256px-jpg/train",
x_col="image_id",
y_col=['Negative for Pneumonia','Typical Appearance','Indeterminate Appearance','Atypical Appearance'],
seed=42,
class_mode="multi_output",
target_size=(256,256))

valid_generator=train_datagen.flow_from_dataframe(
dataframe=train_data_df.loc[5334:],
directory="../input/siim-covid19-resized-to-256px-jpg/train",
x_col="image_id",
y_col=['Negative for Pneumonia','Typical Appearance','Indeterminate Appearance','Atypical Appearance'],
seed=42,
class_mode="multi_output",
target_size=(256,256))
test_datagen=ImageDataGenerator(rescale=1./255.)

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(train_generator[0][0][0], interpolation='nearest')
plt.show()

In [ ]:
import random
from PIL import Image
from numpy import asarray
train_data_df['categorical_train']=pd.to_numeric(train_data_df['categorical_train'])
var_dict2={'Negative_Pneumonia': 1, 'Typical_Appearance': 2, 'Indeterminate_Appearance': 3, 'Atypical_Appearance': 4}
var_dict3={}
jpg_list_train=train_data_df['image_id'].to_list()
batchsize = 100
batch_num=0
numberf=0
X_train_full=np.empty([256, 256])
for i in range(0, len(jpg_list_train), batchsize):
    batch = jpg_list_train[i:i+batchsize]
    batch_num=1+batch_num
    print("Batch Num:", batch_num)
    start=time.time()
    X_train_batch=np.empty([256, 256])
    for i in batch:
        image = Image.open('../input/siim-covid19-resized-to-256px-jpg/train/{}'.format(i))
        data=asarray(image)
        X_train_batch=np.dstack([X_train_batch, data])
        numberf = numberf+1
        clear_output(wait=True)        
    X_train_batch=X_train_batch[0:, 0:, 1:]
    print(X_train_batch.shape)
    X_train_full=np.dstack([X_train_full, X_train_batch])
del X_train_batch
X_train_full=X_train_full[0:, 0:, 1:]
X_train_full=np.transpose(X_train_full, (2, 0, 1))
count = 1
Negative_Pneumonia=random.sample(train_data_df['process_run'][train_data_df['categorical_train']==1].tolist(), 5)
Typical_Appearance=random.sample(train_data_df['process_run'][train_data_df['categorical_train']==2].tolist(), 5)
Indeterminate_Appearance=random.sample(train_data_df['process_run'][train_data_df['categorical_train']==3].tolist(), 5)
Atypical_Appearance=random.sample(train_data_df['process_run'][train_data_df['categorical_train']==4].tolist(), 5)
fig1, [(ax0, ax1, ax2, ax3, ax4), (ax5, ax6, ax7, ax8, ax9), (ax10, ax11, ax12, ax13, ax14), (ax15, ax16, ax17, ax18, ax19)] = plt.subplots(4,5, figsize=(20,15))
fig1.suptitle('Train Selected Results', size='xx-large', y=.92)
ax0.set_ylabel('Negative_Pneumonia', rotation=90, size='large')
ax5.set_ylabel('Typical_Appearance', rotation=90, size='large')
ax10.set_ylabel('Indeterminate_Appearance', rotation=90, size='large')
ax15.set_ylabel('Atypical_Appearance', rotation=90, size='large')
axes1={ax0: Negative_Pneumonia, ax1: Negative_Pneumonia, ax2: Negative_Pneumonia, ax3: Negative_Pneumonia, ax4:Negative_Pneumonia,ax5:Typical_Appearance, ax6:Typical_Appearance, ax7:Typical_Appearance, ax8:Typical_Appearance, ax9:Typical_Appearance,\
       ax10:Indeterminate_Appearance, ax11:Indeterminate_Appearance, ax12:Indeterminate_Appearance, ax13:Indeterminate_Appearance, ax14:Indeterminate_Appearance,ax15:Atypical_Appearance, ax16:Atypical_Appearance, ax17:Atypical_Appearance, ax18:Atypical_Appearance, ax19:Atypical_Appearance}
loopvar=0
for key, value in axes1.items():
    if loopvar==4:
        loopvar=0
    else:
        loopvar=loopvar+1
    key.imshow(Image.fromarray(X_train_full[value[loopvar]]))
del X_train_full
plt.show(fig1)

In [ ]:
### this updated model is the basic way to address the issue while keeping the same simple CONV2D framework
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from matplotlib import ticker
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Convolution2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop, Adam

keras.backend.clear_session()
model=keras.models.Sequential()
model.add(keras.layers.Conv2D(256,3,3,padding = 'same', activation ='relu', input_shape=(256,256,3)))
model.add(keras.layers.Conv2D(256,3,3,padding = "same", activation = "relu"))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2), padding = "same"))
model.add(keras.layers.Conv2D(128,3,3,padding = 'same', activation = 'relu'))
model.add(keras.layers.Conv2D(128,3,3,padding = 'same', activation = 'relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2), padding = "same"))
model.add(keras.layers.Conv2D(64,3,3,padding = 'same', activation = 'relu'))
model.add(keras.layers.Conv2D(64,3,3,padding = 'same', activation = 'relu'))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2), padding = "same"))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256,activation='relu'))
model.add(keras.layers.Dense(64,activation='relu'))
model.add(keras.layers.Dense(4, activation='softmax'))
model.summary()

In [ ]:
from keras.optimizers import Adam
model.compile(loss="sparse_categorical_crossentropy", optimizer='sgd', metrics=[tf.keras.metrics.categorical_accuracy])

In [ ]:
history = model.fit(train_generator, epochs=75, validation_data=(valid_generator))

In [ ]:
pd.DataFrame(history.history)

In [ ]:
pd.DataFrame(history.history).plot(marker='|',markersize=12, figsize=(10,10), linewidth=4)
plt.grid(True)
plt.show()

In [ ]:
jpg_list=os.listdir('../input/siim-covid19-resized-to-256px-jpg/test')
test_data=pd.DataFrame({
    'test': jpg_list
})
test_data

In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
dataframe=test_data,
directory="../input/siim-covid19-resized-to-256px-jpg/test",
x_col="test",
seed=42,
class_mode=None,
target_size=(256,256))

In [ ]:
y_test_pred=model.predict(test_generator)

In [ ]:
y_test_pred

In [ ]:
from numpy import amax
np.argmax(y_test_pred,axis=1)

In [ ]:
y_test_pred=model.predict(test_generator)
y_test_pred = y_test_pred.argmax(axis=-1)
test_meta=pd.read_csv('../input/siim-covid19-resized-to-256px-jpg/meta.csv')
y_predicition=pd.DataFrame({
    'id': jpg_list,
    'y_test_pred': y_test_pred
})
y_predicition.to_csv('y_predicition.csv', index=False)
y_predicition.groupby(['y_test_pred'], as_index=False)[['id']].count()

In [ ]:
import random
y_predicition=y_predicition.reset_index()
y_predicition['y_predicition']=pd.to_numeric(y_predicition['y_predicition'])
var_dict2={'Negative_Pneumonia': 1, 'Typical_Appearance': 2, 'Indeterminate_Appearance': 3, 'Atypical_Appearance': 4}
var_dict3={}
count = 1
for key, value in var_dict2.items():
    list1=y_predicition[y_predicition['y_predicition']==value]
    print(key, list1['index'].to_list())
    print()
    print()
Negative_Pneumonia=random.sample(y_predicition['index'][y_predicition['y_predicition']==1].tolist(), 5)
Typical_Appearance=random.sample(y_predicition['index'][y_predicition['y_predicition']==2].tolist(), 5)
Indeterminate_Appearance=random.sample(y_predicition['index'][y_predicition['y_predicition']==3].tolist(), 5)
Atypical_Appearance=random.sample(y_predicition['index'][y_predicition['y_predicition']==4].tolist(), 5)

In [ ]:
fig, [(ax0, ax1, ax2, ax3, ax4), (ax5, ax6, ax7, ax8, ax9), (ax10, ax11, ax12, ax13, ax14), (ax15, ax16, ax17, ax18, ax19)] = plt.subplots(4,5, figsize=(20,15))
fig.suptitle('Test Selected Results', size='xx-large', y=.92)
ax0.set_ylabel('Negative_Pneumonia', rotation=90, size='large')
ax5.set_ylabel('Typical_Appearance', rotation=90, size='large')
ax10.set_ylabel('Indeterminate_Appearance', rotation=90, size='large')
ax15.set_ylabel('Atypical_Appearance', rotation=90, size='large')
axes1={ax0: Negative_Pneumonia, ax1: Negative_Pneumonia, ax2: Negative_Pneumonia, ax3: Negative_Pneumonia, ax4:Negative_Pneumonia,ax5:Typical_Appearance, ax6:Typical_Appearance, ax7:Typical_Appearance, ax8:Typical_Appearance, ax9:Typical_Appearance,\
       ax10:Indeterminate_Appearance, ax11:Indeterminate_Appearance, ax12:Indeterminate_Appearance, ax13:Indeterminate_Appearance, ax14:Indeterminate_Appearance,ax15:Atypical_Appearance, ax16:Atypical_Appearance, ax17:Atypical_Appearance, ax18:Atypical_Appearance, ax19:Atypical_Appearance}
loopvar=0
for key, value in axes1.items():
    if loopvar==4:
        loopvar=0
    else:
        loopvar=loopvar+1
    key.imshow(train_generator[loopvar][0][0])
rows = ['Row {}'.format(row) for row in ['A', 'B', 'C', 'D']]

plt.show(fig)

In [ ]:
plt.show(fig1)